# Construir banco de Armazenamento Local Hotmart

## Configuração e Importações

### Objetivo: Esse pipeline tem por objetivo normalizar os labels de pesquisa que estejam divergentes entre versões de pesquisa criados.

In [1]:
# Importações básicas
import pandas as pd
import numpy as np
import sys
from pathlib import Path
import os
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), 'src'))

# Adiciona src ao path
sys.path.append('../src')

#

# Utilitários de dados
from data_utils import (
    load_raw_data,
    save_processed_data,
    remove_duplicates,
    handle_missing_values,
    detect_outliers,
    normalize_column,
    process_phone_string,
    process_phone_number,
    clean_and_lower_column,
    flatten_list_to_df
)

CRONOGRAMA_SUBDOMAIN = 'cronogramadosfluentes-xwamel'

# Utilitários SQL
from sql_utils import DatabaseConnection as Dbc, load_query_from_file

# Utilitários de visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Utilitários de API
from api_utils import (
    make_request,
    get_json,
    post_json,
    paginated_request,
    response_to_dataframe
)

# utilitários hotmart
from hotmart_utils import Hotmart

# utilitários tmb
from tmb_utils import TMB   

# Configurações pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Load Database Driver
db = Dbc()

# Inicializar API Hotmart
hotmart = Hotmart()

# Inicializar API TMB
tmb = TMB()

print('✓ Importações concluídas com sucesso!')

✓ Importações concluídas com sucesso!


# Load Buyers Data

#### Load Hotmart Data

In [2]:
df_hotmart_pt1 = load_raw_data('hotmart_pt_1.csv', data_dir='../data/interim')
df_hotmart_pt2 = load_raw_data('hotmart_pt_2.csv', data_dir='../data/interim')
df_hotmart_pt3 = flatten_list_to_df(hotmart.get_sales_history(
    start_date='2025-12-01', end_date='2025-12-31'
))

df_hotmart= pd.concat([df_hotmart_pt1, df_hotmart_pt2, df_hotmart_pt3], ignore_index=True)

FileNotFoundError: [Errno 2] No such file or directory: '../data/interim/hotmart_pt_1.csv'

### Load Hotmart Students


In [ ]:
df_hotmart_students = hotmart.get_students(subdomain=CRONOGRAMA_SUBDOMAIN)

In [ ]:
df_hotmart_students = flatten_list_to_df(df_hotmart_students)

In [ ]:
save_processed_data(df_hotmart_students, 'hotmart_students.csv', data_dir='../data/interim')

### Load TMB Data

In [ ]:
df_tmb = load_raw_data('tmb.csv', sep=';')

"""
Cell generated by Data Wrangler.
"""
def clean_data(df_tmb):
    # Drop columns: 'Produtor', 'Pedido' and 29 other columns
    df_tmb = df_tmb.drop(columns=['Produtor', 'Pedido', 'Endereço completo', 'Logradouro', 'Número', 'Bairro', 'Complemento', 'CEP', 'Estado', 'Cidade', 'País', 'Status', 'Status Financeiro', 'Status Cancelamento', 'Ticket (R$)', 'Modalidade de Contrato', 'Criado Em', 'Data Efetivado', 'Data Cancelado', 'utm_source', 'utm_medium', 'utm_campaign', 'utm_content', 'utm_last_source', 'utm_last_medium', 'utm_last_campaign', 'utm_last_content', 'Renovação automatica', 'Data de Renovação Prevista', 'Renovação enviada', 'Oferta'])
    # Drop column: 'Cliente CPF'
    df_tmb = df_tmb.drop(columns=['Cliente CPF'])
    # Rename column 'Produto' to 'offer_code'
    df_tmb = df_tmb.rename(columns={'Produto': 'offer_id', "Cliente Nome": "name", "Cliente Email": "email", "Telefone": "phone"})

    df_tmb['phone'] = process_phone_string(df_tmb, 'phone')
    
    return df_tmb

df_tmb = clean_data(df_tmb.copy())

,offer_id,name,email,phone
0,COM-BF25-Oferta Premium,ALEXANDRE DA SILVA SANTOS,alexandrechapas23@gmail.com,5585998560317
1,COM-BF25-Oferta Negociação,Josiel Silva Oliveira,josiel_oliveira.380@outlook.com,5562994646038
2,COM-BF25-Oferta Live,Vanderlei tonietto,vanderleitonietto19@gmail.com,5555997192254
3,COM-BF25-Oferta Negociação,Alex Freitas Mota,alexfreitas99@gmail.com,19540560488
4,COM-BF25-Oferta Negociação,Iran Castilho,irancastilho6@gmail.com,5547991332688
